In [1]:
using JuMP
using Gurobi
using Random
using Distributions

In [2]:
N=6 #No. of nodes (1 supplier + 5 DC's)

K=3 #No. of vehicles

Dk=900 #Capacity of vehicles

T=8 #Periods

h=0.025 #Holding cost/unit at each DC

C=[5000 502 488 220 486 742] #Capacity limit for (1 supplier + 5 DC's)

d=[0 0 0 0 0 0 0 0;
75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
] #Demand for each day and (1 supplier + 5 DC's)

c=[
0 140 434 389 419 125;
140 0 300 455 400 97;
434 300 0 609 417 330;
389 455 609 0 256 358;
419 400 417 256 0 316;
125 97 330 358 316 0
] #Transportation cost

std=[
0 0 0 0 0 0 0 0;
11.87 11.87 11.87 11.87 11.87 14.69 14.69 11.87;
10.91 13.89 13.89 13.89 13.89 14.14 11.22 10.91;
11.27 11.27 11.27 11.27 11.27 12.86 12.86 11.27;
13.19 16.03 16.03 16.03 16.03 15.05 11.98 13.19;
13.04 14.00 14.00 14.00 14.00 11.08 9.84 13.04
]# Standard deviation of Demand

I0=[0 98.8 153.9 42.4 23.4 85.7] #Initial inventory
alpha=0.948 # Critical fractile
za=quantile.(Normal(0,1),alpha) #Safety factor
d_new = d + za.*std; #Demand to be satisfied

In [3]:
model=Model(Gurobi.Optimizer)
@variable(model, x[i=1:N,j=1:N,k=1:K,t=1:T], Bin); #If truck k leaves DC i and goes to DC j
@variable(model, y[i=1:N,k=1:K,t=1:T], Bin); #If truck k arrives at DC i
@variable(model, z[j=1:N,k=1:K,t=1:T] >= 0); #Load of truck k when arriving at node j
@variable(model, I[i=1:N,t=0:T] >= 0); #Inventory level at node i till end of period t
@variable(model, q[i=1:N,k=1:K,t=1:T] >= 0); #Quantity delivered to customer i by truck k at start of period t

@objective(model, Min, sum(h*I[i,t] for i=2:N, t=1:T) + sum(c[i,j]*x[i,j,k,t]  for i=1:N, j=1:N, k=1:K, t=1:T))

@constraint(model, [i=2:N], I[i,0] == I0[i]) #Initializing inventory level

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) == I[i,t] + d_new[i,t]); #Balance constraint for each DC

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) <= C[i]); #Capacity constraint for each DC

@constraint(model, [i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); #Constraint which ensures y is 1 when they receive a quantity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i=2:N) <= Dk*y[1,k,t]); #If truck delivers from supplier, it has to activate the supplier

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(i == h ? 0 : x[i,h,k,t] for i = 1:N) == y[h,k,t]); #If truck visits DC

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(j == h ? 0 : x[h,j,k,t] for j = 1:N) == y[h,k,t]); #If truck leaves DC

@constraint(model, [i=2:N,j=2:N,k=1:K,t=1:T], z[i,k,t] - d_new[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d_new)); #To avoid subtours automatically



Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-06


In [4]:
optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x98079532
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+03]
Presolve removed 160 rows and 213 columns
Presolve time: 0.01s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 3.456608e+03, 682 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3456.60754    0   84          - 3456.60754      -     -    0s
H    0     0                    11048.430601 3456.60754  68.7%     -   

In [5]:
if termination_status(model) == MOI.OPTIMAL
   println("RESULTS:")
   println("Objective = $(objective_value(model))")
else
   println("No solution")
end

println();
    for k=1:K
        has_route = false;
        print("Truck:$(value(k))\n")
        for t=1:T
            print("Day:$(value(t))\n")
            for i=1:N
                for j=1:N
                    if (value(x[i,j,k,t]) == 1)
                        print(j,"->",i,"\n")
                        print("$(value(q[i,k,t]))\n")
                        has_route = true;
                    end
                end
            end
        end
        if has_route
            println();
        end
    end

for i=1:N
    for k=1:K
        for t=1:T
            if (value(q[i,k,t]) > 0)
                println(" Node ",i," Truck ",k," Day ", t," Delivery ",value(q[i,k,t]))
            end
        end
    end
end


for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t]))
    end
end


RESULTS:
Objective = 5682.154620591399

Truck:1
Day:1
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
6->1
0.0
1->4
173.57967050980778
4->5
208.57046443149085
5->6
187.0474662770182
Day:8

Truck:2
Day:1
Day:2
6->1
0.0
1->2
409.30780439892317
2->6
490.69219560107683
Day:3
6->1
0.0
1->4
220.0
4->5
485.99999999999864
5->6
194.000000000001
Day:4
Day:5
Day:6
Day:7
Day:8

Truck:3
Day:1
6->1
0.0
5->3
334.1000000000001
1->4
128.6447067256971
4->5
282.4048061457351
3->6
52.81396690392883
Day:2
Day:3
Day:4
Day:5
2->1
0.0
3->2
282.2439922561342
5->3
289.4309303015964
1->4
124.6243772355047
4->5
203.7007002067647
Day:6
Day:7
Day:8

 Node 2 Truck 2 Day 2 Delivery 409.30780439892317
 Node 2 Truck 3 Day 1 Delivery 2.4868995751603507e-12
 Node 2 Truck 3 Day 5 Delivery 282.2439922561342
 Node 3 Truck 3 Day 1 Delivery 334.1000000000001
 Node 3 Truck 3 Day 5 Delivery 289.4309303015964
 Node 4 Truck 1 Day 7 Delivery 173.57967050980778
 Node 4 Truck 2 Day 3 Delivery 220.0
 Node 4 Truck 3 Day 1 Delivery 128.64470672569

In [6]:
#Truck Capacity utilization
for k=1:K
    println("Truck ",k)
    for t=1:T
        println("on day ",t," utilizes ", value(sum(q[i,k,t] for i=1:N))/value(Dk))
    end
end

Truck 1
on day 1 utilizes 0.0
on day 2 utilizes 0.0
on day 3 utilizes 2.4964825583173114e-15
on day 4 utilizes 0.0
on day 5 utilizes 0.0
on day 6 utilizes 0.0
on day 7 utilizes 0.632441779131463
on day 8 utilizes 0.0
Truck 2
on day 1 utilizes 0.0
on day 2 utilizes 1.0
on day 3 utilizes 0.9999999999999997
on day 4 utilizes 0.0
on day 5 utilizes 0.0
on day 6 utilizes 0.0
on day 7 utilizes 0.0
on day 8 utilizes 0.0
Truck 3
on day 1 utilizes 0.8866260886392929
on day 2 utilizes 0.0
on day 3 utilizes 0.0
on day 4 utilizes 0.0
on day 5 utilizes 1.0
on day 6 utilizes 0.0
on day 7 utilizes 0.0
on day 8 utilizes 0.0


In [7]:
#Capacity utilization of the DCs
for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t])/value(C[i]))
    end
end

Day1
Node 2 Inventory Level = 0.0071756745127768634
Node 3 Inventory Level = 0.8347600849512203
Node 4 Inventory Level = 0.3887379698311298
Node 5 Inventory Level = 0.37461108452946246
Node 6 Inventory Level = 0.013361202624592255
Day2
Node 2 Inventory Level = 0.6328927920512921
Node 3 Inventory Level = 0.5942235000438899
Node 4 Inventory Level = 0.0
Node 5 Inventory Level = 0.0
Node 6 Inventory Level = 0.4686597311876806
Day3
Node 2 Inventory Level = 0.44325571756008003
Node 3 Inventory Level = 0.35368691513655615
Node 4 Inventory Level = 0.6112620301688703
Node 5 Inventory Level = 0.6253889154705374
Node 6 Inventory Level = 0.524103548698106
Day4
Node 2 Inventory Level = 0.2536186430688679
Node 3 Inventory Level = 0.11315033022922083
Node 4 Inventory Level = 0.22252406033774208
Node 5 Inventory Level = 0.2507778309410769
Node 6 Inventory Level = 0.31809184060205903
Day5
Node 2 Inventory Level = 0.6262205969763295
Node 3 Inventory Level = 0.4657099139726977
Node 4 Inventory Level = 0.

In [8]:
Random.seed!(0);
counter=zeros(6,8)
#probability=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        new_IL=I0[i];
        for t=1:T
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            new_IL=new_IL+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(new_IL,0),0))/value(actual_demand[i,t]);
            if fillrate>0.95
                counter[i,t]=counter[i,t] + 1
            end
        end
    end
end
probability=counter/10000

6×8 Matrix{Float64}:
 0.0     0.0     0.0  0.0  0.0  0.0  0.0     0.0
 0.992   1.0     1.0  1.0  1.0  1.0  1.0     1.0
 1.0     1.0     1.0  1.0  1.0  1.0  0.9996  1.0
 1.0     0.9949  1.0  1.0  1.0  1.0  1.0     1.0
 1.0     0.9973  1.0  1.0  1.0  1.0  1.0     1.0
 0.9985  1.0     1.0  1.0  1.0  1.0  1.0     1.0

In [9]:
probability # Probability of meeting the fillrate target

6×8 Matrix{Float64}:
 0.0     0.0     0.0  0.0  0.0  0.0  0.0     0.0
 0.992   1.0     1.0  1.0  1.0  1.0  1.0     1.0
 1.0     1.0     1.0  1.0  1.0  1.0  0.9996  1.0
 1.0     0.9949  1.0  1.0  1.0  1.0  1.0     1.0
 1.0     0.9973  1.0  1.0  1.0  1.0  1.0     1.0
 0.9985  1.0     1.0  1.0  1.0  1.0  1.0     1.0